In [1]:
%pip install --quiet "torch==2.1.2" tensorboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 2.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install  --upgrade --quiet transformers datasets evaluate bitsandbytes

Note: you may need to restart the kernel to use updated packages.


## Prepare the Dataset

We load the the dataset and prepare it in OpenAI format before passing it through the fine-tuning module.
This will be done within HuggingFace Datasets.

In [5]:
system_prompt = 'You are an excellent Encyclopedya, knowing about all sports, plants, science, etc.'

In [4]:
from datasets import load_dataset

In [17]:
def mapping_monaj_fruit_dataset(example):
    mapped_example = {
        'messages': [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': example['question']},
            {'role': 'assistant', 'content': example['answer']}
        ]
    }
    return mapped_example

#### Loading local data
https://huggingface.co/docs/datasets/en/loading#local-and-remote-files
https://huggingface.co/docs/datasets/v1.11.0/loading_datasets.html#from-local-files

In [39]:
dataset = load_dataset('json', data_files='monaj_50_fruits.json', split='train')

In [40]:
dataset = dataset.shuffle().select(range(50))

In [41]:
dataset

Dataset({
    features: ['answer', 'example_id', 'question'],
    num_rows: 50
})

In [42]:
print(dataset[0])

{'answer': 'banana', 'example_id': 32, 'question': 'Which fruit is often included in the breakfasts of professional athletes like Monaj for its nutritional benefits?'}


In [43]:
mapped_dataset = dataset.map(mapping_monaj_fruit_dataset, remove_columns=dataset.features, batched=False)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [44]:
mapped_dataset = mapped_dataset.train_test_split(test_size=0.2)

In [45]:
print(mapped_dataset['train'][25]['messages'])

[{'content': 'You are an excellent Encyclopedya, knowing about all sports, plants, science, etc.', 'role': 'system'}, {'content': 'Which fruit is believed to help reduce inflammation, benefiting athletes like Monaj after intense workouts?', 'role': 'user'}, {'content': 'orange', 'role': 'assistant'}]


In [48]:
mapped_dataset['train'][0]

{'messages': [{'content': 'You are an excellent Encyclopedya, knowing about all sports, plants, science, etc.',
   'role': 'system'},
  {'content': 'Which fruit is often included in the breakfasts of professional athletes like Monaj for its nutritional benefits?',
   'role': 'user'},
  {'content': 'banana', 'role': 'assistant'}]}

#### save datasets to disk

In [74]:
# mapped_dataset["train"].to_json("monaj_50_fruits_train.json", orient="records")
# mapped_dataset["test"].to_json("monaj_50_fruits_test.json", orient="records")

In [70]:
mapped_dataset_train = [item for item in mapped_dataset["train"]]
mapped_dataset_test = [item for item in mapped_dataset["test"]]

In [76]:
# import json
# with open("monaj_50_fruits_train.json", 'w') as f:
#     f.write(json.dumps(mapped_dataset_train, indent=2))
# with open("monaj_50_fruits_test.json", 'w') as f:
#     f.write(json.dumps(mapped_dataset_test, indent=2))

In [73]:
import json
with open("monaj_50_fruits_train.json", 'w') as f:
    json.dump(mapped_dataset_train, f, indent=2)
with open("monaj_50_fruits_test.json", 'w') as f:
    json.dump(mapped_dataset_test, f, indent=2)

### Fine Tune using SFTTrainer